In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import pearsonr
import time

In [2]:
# Load the original neural and word2vec similarity matrices
word2vec_sim_df = pd.read_csv('matrix_residuals_semdist15.csv')
neural_sim_df = pd.read_csv('p1_sim.csv')

#Align the columns of the neural similarity matrix to match the word2vec similarity matrix
neural_sim_df_aligned = neural_sim_df[word2vec_sim_df.columns]

# Reorder the rows of the neural similarity matrix to match the order in the word2vec similarity matrix
neural_sim_df_reordered = neural_sim_df_aligned.set_index('term').reindex(word2vec_sim_df['term']).reset_index()
# List of words (excluding the 'term' column)
words = word2vec_sim_df.columns[1:]

# Initialize a list to store results
decoding_results = []

In [3]:
neural_sim_df_reordered

,term,horse,bear,cat,dog,cow,leg,arm,hand,foot,...,tomato,celery,corn,carrot,lettuce,bicycle,car,train,truck,airplane
0,horse,1.000000,0.867668,0.794402,0.817715,0.762252,0.655147,0.560490,0.798085,0.773818,...,0.679341,0.517919,0.809874,0.586457,0.814620,0.886597,0.591752,0.698553,0.770612,0.830323
1,bear,0.867668,1.000000,0.784209,0.856424,0.842841,0.576497,0.529558,0.703956,0.695968,...,0.678013,0.514908,0.710625,0.626575,0.799161,0.849823,0.696179,0.748991,0.761983,0.828290
2,cat,0.794402,0.784209,1.000000,0.801168,0.706735,0.798449,0.518258,0.803878,0.672096,...,0.571532,0.317849,0.771336,0.417673,0.768428,0.835674,0.513325,0.669761,0.608832,0.677270
3,dog,0.817715,0.856424,0.801168,1.000000,0.869596,0.669203,0.587157,0.763389,0.744822,...,0.644053,0.475084,0.698746,0.537472,0.784360,0.831094,0.610807,0.618325,0.649231,0.724007
4,cow,0.762252,0.842841,0.706735,0.869596,1.000000,0.564517,0.602313,0.636055,0.691362,...,0.561507,0.544593,0.655788,0.623611,0.740062,0.792543,0.700272,0.650916,0.690289,0.769298
5,leg,0.655147,0.576497,0.798449,0.669203,0.564517,1.000000,0.589304,0.745915,0.706871,...,0.324236,0.241527,0.740154,0.298281,0.635136,0.779629,0.405805,0.586700,0.483917,0.521596
6,arm,0.560490,0.529558,0.518258,0.587157,0.602313,0.589304,1.000000,0.683080,0.846284,...,0.211426,0.550446,0.497993,0.652083,0.436008,0.698630,0.389628,0.465599,0.566975,0.611928
7,hand,0.798085,0.703956,0.803878,0.763389,0.636055,0.745915,0.683080,1.000000,0.840091,...,0.528932,0.426071,0.744326,0.466841,0.739569,0.838479,0.416598,0.552607,0.609400,0.669398
8,foot,0.773818,0.695968,0.672096,0.744822,0.691362,0.706871,0.846284,0.840091,1.000000,...,0.449388,0.578857,0.695290,0.649962,0.644539,0.824975,0.479248,0.553614,0.694302,0.709507
9,eye,0.811312,0.797959,0.739001,0.761645,0.764978,0.678523,0.614393,0.756584,0.762380,...,0.518579,0.535730,0.842577,0.609926,0.822865,0.888571,0.742480,0.823249,0.822879,0.826608


In [4]:
neural_sim_df_reordered

,term,horse,bear,cat,dog,cow,leg,arm,hand,foot,...,tomato,celery,corn,carrot,lettuce,bicycle,car,train,truck,airplane
0,horse,1.000000,0.867668,0.794402,0.817715,0.762252,0.655147,0.560490,0.798085,0.773818,...,0.679341,0.517919,0.809874,0.586457,0.814620,0.886597,0.591752,0.698553,0.770612,0.830323
1,bear,0.867668,1.000000,0.784209,0.856424,0.842841,0.576497,0.529558,0.703956,0.695968,...,0.678013,0.514908,0.710625,0.626575,0.799161,0.849823,0.696179,0.748991,0.761983,0.828290
2,cat,0.794402,0.784209,1.000000,0.801168,0.706735,0.798449,0.518258,0.803878,0.672096,...,0.571532,0.317849,0.771336,0.417673,0.768428,0.835674,0.513325,0.669761,0.608832,0.677270
3,dog,0.817715,0.856424,0.801168,1.000000,0.869596,0.669203,0.587157,0.763389,0.744822,...,0.644053,0.475084,0.698746,0.537472,0.784360,0.831094,0.610807,0.618325,0.649231,0.724007
4,cow,0.762252,0.842841,0.706735,0.869596,1.000000,0.564517,0.602313,0.636055,0.691362,...,0.561507,0.544593,0.655788,0.623611,0.740062,0.792543,0.700272,0.650916,0.690289,0.769298
5,leg,0.655147,0.576497,0.798449,0.669203,0.564517,1.000000,0.589304,0.745915,0.706871,...,0.324236,0.241527,0.740154,0.298281,0.635136,0.779629,0.405805,0.586700,0.483917,0.521596
6,arm,0.560490,0.529558,0.518258,0.587157,0.602313,0.589304,1.000000,0.683080,0.846284,...,0.211426,0.550446,0.497993,0.652083,0.436008,0.698630,0.389628,0.465599,0.566975,0.611928
7,hand,0.798085,0.703956,0.803878,0.763389,0.636055,0.745915,0.683080,1.000000,0.840091,...,0.528932,0.426071,0.744326,0.466841,0.739569,0.838479,0.416598,0.552607,0.609400,0.669398
8,foot,0.773818,0.695968,0.672096,0.744822,0.691362,0.706871,0.846284,0.840091,1.000000,...,0.449388,0.578857,0.695290,0.649962,0.644539,0.824975,0.479248,0.553614,0.694302,0.709507
9,eye,0.811312,0.797959,0.739001,0.761645,0.764978,0.678523,0.614393,0.756584,0.762380,...,0.518579,0.535730,0.842577,0.609926,0.822865,0.888571,0.742480,0.823249,0.822879,0.826608


In [5]:
# Iterate over all unique pairs of words
for word1, word2 in combinations(words, 2):
    # Extract vectors
    word1_neural_vector = neural_sim_df_reordered[word1]
    word2_neural_vector = neural_sim_df_reordered[word2]
    word1_semantic_vector = word2vec_sim_df[word1]
    word2_semantic_vector = word2vec_sim_df[word2]

    # Remove indices with perfect correlations
    perfect_corr_indices = (word1_neural_vector == 1.0) | (word2_neural_vector == 1.0) | \
                           (word1_semantic_vector == 1.0) | (word2_semantic_vector == 1.0)
    word1_neural_vector_filtered = word1_neural_vector[~perfect_corr_indices]
    word2_neural_vector_filtered = word2_neural_vector[~perfect_corr_indices]
    word1_semantic_vector_filtered = word1_semantic_vector[~perfect_corr_indices]
    word2_semantic_vector_filtered = word2_semantic_vector[~perfect_corr_indices]

    # Calculate correlations
    corr_word1_neural_semantic = pearsonr(word1_neural_vector_filtered, word1_semantic_vector_filtered)[0]
    corr_word1_neural_word2_semantic = pearsonr(word1_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_semantic = pearsonr(word2_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_word1_semantic = pearsonr(word2_neural_vector_filtered, word1_semantic_vector_filtered)[0]

    # Check decoding accuracy
    decode_accuracy_word1 = corr_word1_neural_semantic > corr_word1_neural_word2_semantic
    decode_accuracy_word2 = corr_word2_neural_semantic > corr_word2_neural_word1_semantic

    # Append results
    decoding_results.append({
        'word1': word1,
        'word2': word2,
        'corr_word1_neural_semantic': corr_word1_neural_semantic,
        'corr_word1_neural_word2_semantic': corr_word1_neural_word2_semantic,
        'corr_word2_neural_semantic': corr_word2_neural_semantic,
        'corr_word2_neural_word1_semantic': corr_word2_neural_word1_semantic,
        'decode_accuracy_word1': decode_accuracy_word1,
        'decode_accuracy_word2': decode_accuracy_word2
    })

# Convert results to a DataFrame
decoding_results_df = pd.DataFrame(decoding_results)

# Save the results to a CSV file
#decoding_results_df.to_csv('decoding_results.csv', index=False)


In [7]:
decoding_results_df

,word1,word2,corr_word1_neural_semantic,corr_word1_neural_word2_semantic,corr_word2_neural_semantic,corr_word2_neural_word1_semantic,decode_accuracy_word1,decode_accuracy_word2
0,horse,bear,0.167891,0.155666,0.144818,0.143094,True,True
1,horse,cat,0.153609,0.135756,0.060013,0.162162,True,False
2,horse,dog,0.149693,0.081105,-0.098696,0.028098,True,False
3,horse,cow,0.169983,-0.037443,-0.070567,0.045873,True,False
4,horse,leg,0.148082,0.191753,0.143401,0.169095,False,False
...,...,...,...,...,...,...,...,...
1765,car,truck,0.052116,0.372079,0.385075,0.095510,False,True
1766,car,airplane,0.013257,0.438265,0.410750,0.112438,False,True
1767,train,truck,0.418073,0.394883,0.355418,0.427436,True,False
1768,train,airplane,0.415714,0.472863,0.404492,0.463016,False,False


In [6]:
# Calculate the overall decoder accuracy from the results DataFrame
total_accuracy = (decoding_results_df['decode_accuracy_word1'].sum() + 
                  decoding_results_df['decode_accuracy_word2'].sum())
p1_overall_accuracy = total_accuracy / (2 * len(decoding_results_df))




In [7]:
neural_sim_df = pd.read_csv('p2_sim.csv')

#Align the columns of the neural similarity matrix to match the word2vec similarity matrix
neural_sim_df_aligned = neural_sim_df[word2vec_sim_df.columns]

# Reorder the rows of the neural similarity matrix to match the order in the word2vec similarity matrix
neural_sim_df_reordered = neural_sim_df_aligned.set_index('term').reindex(word2vec_sim_df['term']).reset_index()
# List of words (excluding the 'term' column)
words = word2vec_sim_df.columns[1:]

# Initialize a list to store results
decoding_results = []

# Iterate over all unique pairs of words
for word1, word2 in combinations(words, 2):
    # Extract vectors
    word1_neural_vector = neural_sim_df_reordered[word1]
    word2_neural_vector = neural_sim_df_reordered[word2]
    word1_semantic_vector = word2vec_sim_df[word1]
    word2_semantic_vector = word2vec_sim_df[word2]

    # Remove indices with perfect correlations
    perfect_corr_indices = (word1_neural_vector == 1.0) | (word2_neural_vector == 1.0) | \
                           (word1_semantic_vector == 1.0) | (word2_semantic_vector == 1.0)
    word1_neural_vector_filtered = word1_neural_vector[~perfect_corr_indices]
    word2_neural_vector_filtered = word2_neural_vector[~perfect_corr_indices]
    word1_semantic_vector_filtered = word1_semantic_vector[~perfect_corr_indices]
    word2_semantic_vector_filtered = word2_semantic_vector[~perfect_corr_indices]

    # Calculate correlations
    corr_word1_neural_semantic = pearsonr(word1_neural_vector_filtered, word1_semantic_vector_filtered)[0]
    corr_word1_neural_word2_semantic = pearsonr(word1_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_semantic = pearsonr(word2_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_word1_semantic = pearsonr(word2_neural_vector_filtered, word1_semantic_vector_filtered)[0]

    # Check decoding accuracy
    decode_accuracy_word1 = corr_word1_neural_semantic > corr_word1_neural_word2_semantic
    decode_accuracy_word2 = corr_word2_neural_semantic > corr_word2_neural_word1_semantic

    # Append results
    decoding_results.append({
        'word1': word1,
        'word2': word2,
        'corr_word1_neural_semantic': corr_word1_neural_semantic,
        'corr_word1_neural_word2_semantic': corr_word1_neural_word2_semantic,
        'corr_word2_neural_semantic': corr_word2_neural_semantic,
        'corr_word2_neural_word1_semantic': corr_word2_neural_word1_semantic,
        'decode_accuracy_word1': decode_accuracy_word1,
        'decode_accuracy_word2': decode_accuracy_word2
    })

# Convert results to a DataFrame
decoding_results_df = pd.DataFrame(decoding_results)

# Calculate the overall decoder accuracy from the results DataFrame
total_accuracy = (decoding_results_df['decode_accuracy_word1'].sum() + 
                  decoding_results_df['decode_accuracy_word2'].sum())
p2_overall_accuracy = total_accuracy / (2 * len(decoding_results_df))



In [8]:
neural_sim_df = pd.read_csv('p3_sim.csv')

#Align the columns of the neural similarity matrix to match the word2vec similarity matrix
neural_sim_df_aligned = neural_sim_df[word2vec_sim_df.columns]

# Reorder the rows of the neural similarity matrix to match the order in the word2vec similarity matrix
neural_sim_df_reordered = neural_sim_df_aligned.set_index('term').reindex(word2vec_sim_df['term']).reset_index()
# List of words (excluding the 'term' column)
words = word2vec_sim_df.columns[1:]

# Initialize a list to store results
decoding_results = []

# Iterate over all unique pairs of words
for word1, word2 in combinations(words, 2):
    # Extract vectors
    word1_neural_vector = neural_sim_df_reordered[word1]
    word2_neural_vector = neural_sim_df_reordered[word2]
    word1_semantic_vector = word2vec_sim_df[word1]
    word2_semantic_vector = word2vec_sim_df[word2]

    # Remove indices with perfect correlations
    perfect_corr_indices = (word1_neural_vector == 1.0) | (word2_neural_vector == 1.0) | \
                           (word1_semantic_vector == 1.0) | (word2_semantic_vector == 1.0)
    word1_neural_vector_filtered = word1_neural_vector[~perfect_corr_indices]
    word2_neural_vector_filtered = word2_neural_vector[~perfect_corr_indices]
    word1_semantic_vector_filtered = word1_semantic_vector[~perfect_corr_indices]
    word2_semantic_vector_filtered = word2_semantic_vector[~perfect_corr_indices]

    # Calculate correlations
    corr_word1_neural_semantic = pearsonr(word1_neural_vector_filtered, word1_semantic_vector_filtered)[0]
    corr_word1_neural_word2_semantic = pearsonr(word1_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_semantic = pearsonr(word2_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_word1_semantic = pearsonr(word2_neural_vector_filtered, word1_semantic_vector_filtered)[0]

    # Check decoding accuracy
    decode_accuracy_word1 = corr_word1_neural_semantic > corr_word1_neural_word2_semantic
    decode_accuracy_word2 = corr_word2_neural_semantic > corr_word2_neural_word1_semantic

    # Append results
    decoding_results.append({
        'word1': word1,
        'word2': word2,
        'corr_word1_neural_semantic': corr_word1_neural_semantic,
        'corr_word1_neural_word2_semantic': corr_word1_neural_word2_semantic,
        'corr_word2_neural_semantic': corr_word2_neural_semantic,
        'corr_word2_neural_word1_semantic': corr_word2_neural_word1_semantic,
        'decode_accuracy_word1': decode_accuracy_word1,
        'decode_accuracy_word2': decode_accuracy_word2
    })

# Convert results to a DataFrame
decoding_results_df = pd.DataFrame(decoding_results)

# Calculate the overall decoder accuracy from the results DataFrame
total_accuracy = (decoding_results_df['decode_accuracy_word1'].sum() + 
                  decoding_results_df['decode_accuracy_word2'].sum())
p3_overall_accuracy = total_accuracy / (2 * len(decoding_results_df))



In [9]:
neural_sim_df = pd.read_csv('p4_sim.csv')

#Align the columns of the neural similarity matrix to match the word2vec similarity matrix
neural_sim_df_aligned = neural_sim_df[word2vec_sim_df.columns]

# Reorder the rows of the neural similarity matrix to match the order in the word2vec similarity matrix
neural_sim_df_reordered = neural_sim_df_aligned.set_index('term').reindex(word2vec_sim_df['term']).reset_index()
# List of words (excluding the 'term' column)
words = word2vec_sim_df.columns[1:]

# Initialize a list to store results
decoding_results = []

# Iterate over all unique pairs of words
for word1, word2 in combinations(words, 2):
    # Extract vectors
    word1_neural_vector = neural_sim_df_reordered[word1]
    word2_neural_vector = neural_sim_df_reordered[word2]
    word1_semantic_vector = word2vec_sim_df[word1]
    word2_semantic_vector = word2vec_sim_df[word2]

    # Remove indices with perfect correlations
    perfect_corr_indices = (word1_neural_vector == 1.0) | (word2_neural_vector == 1.0) | \
                           (word1_semantic_vector == 1.0) | (word2_semantic_vector == 1.0)
    word1_neural_vector_filtered = word1_neural_vector[~perfect_corr_indices]
    word2_neural_vector_filtered = word2_neural_vector[~perfect_corr_indices]
    word1_semantic_vector_filtered = word1_semantic_vector[~perfect_corr_indices]
    word2_semantic_vector_filtered = word2_semantic_vector[~perfect_corr_indices]

    # Calculate correlations
    corr_word1_neural_semantic = pearsonr(word1_neural_vector_filtered, word1_semantic_vector_filtered)[0]
    corr_word1_neural_word2_semantic = pearsonr(word1_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_semantic = pearsonr(word2_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_word1_semantic = pearsonr(word2_neural_vector_filtered, word1_semantic_vector_filtered)[0]

    # Check decoding accuracy
    decode_accuracy_word1 = corr_word1_neural_semantic > corr_word1_neural_word2_semantic
    decode_accuracy_word2 = corr_word2_neural_semantic > corr_word2_neural_word1_semantic

    # Append results
    decoding_results.append({
        'word1': word1,
        'word2': word2,
        'corr_word1_neural_semantic': corr_word1_neural_semantic,
        'corr_word1_neural_word2_semantic': corr_word1_neural_word2_semantic,
        'corr_word2_neural_semantic': corr_word2_neural_semantic,
        'corr_word2_neural_word1_semantic': corr_word2_neural_word1_semantic,
        'decode_accuracy_word1': decode_accuracy_word1,
        'decode_accuracy_word2': decode_accuracy_word2
    })

# Convert results to a DataFrame
decoding_results_df = pd.DataFrame(decoding_results)

# Calculate the overall decoder accuracy from the results DataFrame
total_accuracy = (decoding_results_df['decode_accuracy_word1'].sum() + 
                  decoding_results_df['decode_accuracy_word2'].sum())
p4_overall_accuracy = total_accuracy / (2 * len(decoding_results_df))



In [10]:
neural_sim_df = pd.read_csv('p5_sim.csv')

#Align the columns of the neural similarity matrix to match the word2vec similarity matrix
neural_sim_df_aligned = neural_sim_df[word2vec_sim_df.columns]

# Reorder the rows of the neural similarity matrix to match the order in the word2vec similarity matrix
neural_sim_df_reordered = neural_sim_df_aligned.set_index('term').reindex(word2vec_sim_df['term']).reset_index()
# List of words (excluding the 'term' column)
words = word2vec_sim_df.columns[1:]

# Initialize a list to store results
decoding_results = []

# Iterate over all unique pairs of words
for word1, word2 in combinations(words, 2):
    # Extract vectors
    word1_neural_vector = neural_sim_df_reordered[word1]
    word2_neural_vector = neural_sim_df_reordered[word2]
    word1_semantic_vector = word2vec_sim_df[word1]
    word2_semantic_vector = word2vec_sim_df[word2]

    # Remove indices with perfect correlations
    perfect_corr_indices = (word1_neural_vector == 1.0) | (word2_neural_vector == 1.0) | \
                           (word1_semantic_vector == 1.0) | (word2_semantic_vector == 1.0)
    word1_neural_vector_filtered = word1_neural_vector[~perfect_corr_indices]
    word2_neural_vector_filtered = word2_neural_vector[~perfect_corr_indices]
    word1_semantic_vector_filtered = word1_semantic_vector[~perfect_corr_indices]
    word2_semantic_vector_filtered = word2_semantic_vector[~perfect_corr_indices]

    # Calculate correlations
    corr_word1_neural_semantic = pearsonr(word1_neural_vector_filtered, word1_semantic_vector_filtered)[0]
    corr_word1_neural_word2_semantic = pearsonr(word1_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_semantic = pearsonr(word2_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_word1_semantic = pearsonr(word2_neural_vector_filtered, word1_semantic_vector_filtered)[0]

    # Check decoding accuracy
    decode_accuracy_word1 = corr_word1_neural_semantic > corr_word1_neural_word2_semantic
    decode_accuracy_word2 = corr_word2_neural_semantic > corr_word2_neural_word1_semantic

    # Append results
    decoding_results.append({
        'word1': word1,
        'word2': word2,
        'corr_word1_neural_semantic': corr_word1_neural_semantic,
        'corr_word1_neural_word2_semantic': corr_word1_neural_word2_semantic,
        'corr_word2_neural_semantic': corr_word2_neural_semantic,
        'corr_word2_neural_word1_semantic': corr_word2_neural_word1_semantic,
        'decode_accuracy_word1': decode_accuracy_word1,
        'decode_accuracy_word2': decode_accuracy_word2
    })

# Convert results to a DataFrame
decoding_results_df = pd.DataFrame(decoding_results)

# Calculate the overall decoder accuracy from the results DataFrame
total_accuracy = (decoding_results_df['decode_accuracy_word1'].sum() + 
                  decoding_results_df['decode_accuracy_word2'].sum())
p5_overall_accuracy = total_accuracy / (2 * len(decoding_results_df))



In [11]:
neural_sim_df = pd.read_csv('p6_sim.csv')

#Align the columns of the neural similarity matrix to match the word2vec similarity matrix
neural_sim_df_aligned = neural_sim_df[word2vec_sim_df.columns]

# Reorder the rows of the neural similarity matrix to match the order in the word2vec similarity matrix
neural_sim_df_reordered = neural_sim_df_aligned.set_index('term').reindex(word2vec_sim_df['term']).reset_index()
# List of words (excluding the 'term' column)
words = word2vec_sim_df.columns[1:]

# Initialize a list to store results
decoding_results = []

# Iterate over all unique pairs of words
for word1, word2 in combinations(words, 2):
    # Extract vectors
    word1_neural_vector = neural_sim_df_reordered[word1]
    word2_neural_vector = neural_sim_df_reordered[word2]
    word1_semantic_vector = word2vec_sim_df[word1]
    word2_semantic_vector = word2vec_sim_df[word2]

    # Remove indices with perfect correlations
    perfect_corr_indices = (word1_neural_vector == 1.0) | (word2_neural_vector == 1.0) | \
                           (word1_semantic_vector == 1.0) | (word2_semantic_vector == 1.0)
    word1_neural_vector_filtered = word1_neural_vector[~perfect_corr_indices]
    word2_neural_vector_filtered = word2_neural_vector[~perfect_corr_indices]
    word1_semantic_vector_filtered = word1_semantic_vector[~perfect_corr_indices]
    word2_semantic_vector_filtered = word2_semantic_vector[~perfect_corr_indices]

    # Calculate correlations
    corr_word1_neural_semantic = pearsonr(word1_neural_vector_filtered, word1_semantic_vector_filtered)[0]
    corr_word1_neural_word2_semantic = pearsonr(word1_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_semantic = pearsonr(word2_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_word1_semantic = pearsonr(word2_neural_vector_filtered, word1_semantic_vector_filtered)[0]

    # Check decoding accuracy
    decode_accuracy_word1 = corr_word1_neural_semantic > corr_word1_neural_word2_semantic
    decode_accuracy_word2 = corr_word2_neural_semantic > corr_word2_neural_word1_semantic

    # Append results
    decoding_results.append({
        'word1': word1,
        'word2': word2,
        'corr_word1_neural_semantic': corr_word1_neural_semantic,
        'corr_word1_neural_word2_semantic': corr_word1_neural_word2_semantic,
        'corr_word2_neural_semantic': corr_word2_neural_semantic,
        'corr_word2_neural_word1_semantic': corr_word2_neural_word1_semantic,
        'decode_accuracy_word1': decode_accuracy_word1,
        'decode_accuracy_word2': decode_accuracy_word2
    })

# Convert results to a DataFrame
decoding_results_df = pd.DataFrame(decoding_results)

# Calculate the overall decoder accuracy from the results DataFrame
total_accuracy = (decoding_results_df['decode_accuracy_word1'].sum() + 
                  decoding_results_df['decode_accuracy_word2'].sum())
p6_overall_accuracy = total_accuracy / (2 * len(decoding_results_df))



In [12]:
neural_sim_df = pd.read_csv('p7_sim.csv')

#Align the columns of the neural similarity matrix to match the word2vec similarity matrix
neural_sim_df_aligned = neural_sim_df[word2vec_sim_df.columns]

# Reorder the rows of the neural similarity matrix to match the order in the word2vec similarity matrix
neural_sim_df_reordered = neural_sim_df_aligned.set_index('term').reindex(word2vec_sim_df['term']).reset_index()
# List of words (excluding the 'term' column)
words = word2vec_sim_df.columns[1:]

# Initialize a list to store results
decoding_results = []

# Iterate over all unique pairs of words
for word1, word2 in combinations(words, 2):
    # Extract vectors
    word1_neural_vector = neural_sim_df_reordered[word1]
    word2_neural_vector = neural_sim_df_reordered[word2]
    word1_semantic_vector = word2vec_sim_df[word1]
    word2_semantic_vector = word2vec_sim_df[word2]

    # Remove indices with perfect correlations
    perfect_corr_indices = (word1_neural_vector == 1.0) | (word2_neural_vector == 1.0) | \
                           (word1_semantic_vector == 1.0) | (word2_semantic_vector == 1.0)
    word1_neural_vector_filtered = word1_neural_vector[~perfect_corr_indices]
    word2_neural_vector_filtered = word2_neural_vector[~perfect_corr_indices]
    word1_semantic_vector_filtered = word1_semantic_vector[~perfect_corr_indices]
    word2_semantic_vector_filtered = word2_semantic_vector[~perfect_corr_indices]

    # Calculate correlations
    corr_word1_neural_semantic = pearsonr(word1_neural_vector_filtered, word1_semantic_vector_filtered)[0]
    corr_word1_neural_word2_semantic = pearsonr(word1_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_semantic = pearsonr(word2_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_word1_semantic = pearsonr(word2_neural_vector_filtered, word1_semantic_vector_filtered)[0]

    # Check decoding accuracy
    decode_accuracy_word1 = corr_word1_neural_semantic > corr_word1_neural_word2_semantic
    decode_accuracy_word2 = corr_word2_neural_semantic > corr_word2_neural_word1_semantic

    # Append results
    decoding_results.append({
        'word1': word1,
        'word2': word2,
        'corr_word1_neural_semantic': corr_word1_neural_semantic,
        'corr_word1_neural_word2_semantic': corr_word1_neural_word2_semantic,
        'corr_word2_neural_semantic': corr_word2_neural_semantic,
        'corr_word2_neural_word1_semantic': corr_word2_neural_word1_semantic,
        'decode_accuracy_word1': decode_accuracy_word1,
        'decode_accuracy_word2': decode_accuracy_word2
    })

# Convert results to a DataFrame
decoding_results_df = pd.DataFrame(decoding_results)

# Calculate the overall decoder accuracy from the results DataFrame
total_accuracy = (decoding_results_df['decode_accuracy_word1'].sum() + 
                  decoding_results_df['decode_accuracy_word2'].sum())
p7_overall_accuracy = total_accuracy / (2 * len(decoding_results_df))



In [13]:
neural_sim_df = pd.read_csv('p8_sim.csv')

#Align the columns of the neural similarity matrix to match the word2vec similarity matrix
neural_sim_df_aligned = neural_sim_df[word2vec_sim_df.columns]

# Reorder the rows of the neural similarity matrix to match the order in the word2vec similarity matrix
neural_sim_df_reordered = neural_sim_df_aligned.set_index('term').reindex(word2vec_sim_df['term']).reset_index()
# List of words (excluding the 'term' column)
words = word2vec_sim_df.columns[1:]

# Initialize a list to store results
decoding_results = []

# Iterate over all unique pairs of words
for word1, word2 in combinations(words, 2):
    # Extract vectors
    word1_neural_vector = neural_sim_df_reordered[word1]
    word2_neural_vector = neural_sim_df_reordered[word2]
    word1_semantic_vector = word2vec_sim_df[word1]
    word2_semantic_vector = word2vec_sim_df[word2]

    # Remove indices with perfect correlations
    perfect_corr_indices = (word1_neural_vector == 1.0) | (word2_neural_vector == 1.0) | \
                           (word1_semantic_vector == 1.0) | (word2_semantic_vector == 1.0)
    word1_neural_vector_filtered = word1_neural_vector[~perfect_corr_indices]
    word2_neural_vector_filtered = word2_neural_vector[~perfect_corr_indices]
    word1_semantic_vector_filtered = word1_semantic_vector[~perfect_corr_indices]
    word2_semantic_vector_filtered = word2_semantic_vector[~perfect_corr_indices]

    # Calculate correlations
    corr_word1_neural_semantic = pearsonr(word1_neural_vector_filtered, word1_semantic_vector_filtered)[0]
    corr_word1_neural_word2_semantic = pearsonr(word1_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_semantic = pearsonr(word2_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_word1_semantic = pearsonr(word2_neural_vector_filtered, word1_semantic_vector_filtered)[0]

    # Check decoding accuracy
    decode_accuracy_word1 = corr_word1_neural_semantic > corr_word1_neural_word2_semantic
    decode_accuracy_word2 = corr_word2_neural_semantic > corr_word2_neural_word1_semantic

    # Append results
    decoding_results.append({
        'word1': word1,
        'word2': word2,
        'corr_word1_neural_semantic': corr_word1_neural_semantic,
        'corr_word1_neural_word2_semantic': corr_word1_neural_word2_semantic,
        'corr_word2_neural_semantic': corr_word2_neural_semantic,
        'corr_word2_neural_word1_semantic': corr_word2_neural_word1_semantic,
        'decode_accuracy_word1': decode_accuracy_word1,
        'decode_accuracy_word2': decode_accuracy_word2
    })

# Convert results to a DataFrame
decoding_results_df = pd.DataFrame(decoding_results)

# Calculate the overall decoder accuracy from the results DataFrame
total_accuracy = (decoding_results_df['decode_accuracy_word1'].sum() + 
                  decoding_results_df['decode_accuracy_word2'].sum())
p8_overall_accuracy = total_accuracy / (2 * len(decoding_results_df))



In [14]:
neural_sim_df = pd.read_csv('p9_sim.csv')

#Align the columns of the neural similarity matrix to match the word2vec similarity matrix
neural_sim_df_aligned = neural_sim_df[word2vec_sim_df.columns]

# Reorder the rows of the neural similarity matrix to match the order in the word2vec similarity matrix
neural_sim_df_reordered = neural_sim_df_aligned.set_index('term').reindex(word2vec_sim_df['term']).reset_index()
# List of words (excluding the 'term' column)
words = word2vec_sim_df.columns[1:]

# Initialize a list to store results
decoding_results = []

# Iterate over all unique pairs of words
for word1, word2 in combinations(words, 2):
    # Extract vectors
    word1_neural_vector = neural_sim_df_reordered[word1]
    word2_neural_vector = neural_sim_df_reordered[word2]
    word1_semantic_vector = word2vec_sim_df[word1]
    word2_semantic_vector = word2vec_sim_df[word2]

    # Remove indices with perfect correlations
    perfect_corr_indices = (word1_neural_vector == 1.0) | (word2_neural_vector == 1.0) | \
                           (word1_semantic_vector == 1.0) | (word2_semantic_vector == 1.0)
    word1_neural_vector_filtered = word1_neural_vector[~perfect_corr_indices]
    word2_neural_vector_filtered = word2_neural_vector[~perfect_corr_indices]
    word1_semantic_vector_filtered = word1_semantic_vector[~perfect_corr_indices]
    word2_semantic_vector_filtered = word2_semantic_vector[~perfect_corr_indices]

    # Calculate correlations
    corr_word1_neural_semantic = pearsonr(word1_neural_vector_filtered, word1_semantic_vector_filtered)[0]
    corr_word1_neural_word2_semantic = pearsonr(word1_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_semantic = pearsonr(word2_neural_vector_filtered, word2_semantic_vector_filtered)[0]
    corr_word2_neural_word1_semantic = pearsonr(word2_neural_vector_filtered, word1_semantic_vector_filtered)[0]

    # Check decoding accuracy
    decode_accuracy_word1 = corr_word1_neural_semantic > corr_word1_neural_word2_semantic
    decode_accuracy_word2 = corr_word2_neural_semantic > corr_word2_neural_word1_semantic

    # Append results
    decoding_results.append({
        'word1': word1,
        'word2': word2,
        'corr_word1_neural_semantic': corr_word1_neural_semantic,
        'corr_word1_neural_word2_semantic': corr_word1_neural_word2_semantic,
        'corr_word2_neural_semantic': corr_word2_neural_semantic,
        'corr_word2_neural_word1_semantic': corr_word2_neural_word1_semantic,
        'decode_accuracy_word1': decode_accuracy_word1,
        'decode_accuracy_word2': decode_accuracy_word2
    })

# Convert results to a DataFrame
decoding_results_df = pd.DataFrame(decoding_results)

# Calculate the overall decoder accuracy from the results DataFrame
total_accuracy = (decoding_results_df['decode_accuracy_word1'].sum() + 
                  decoding_results_df['decode_accuracy_word2'].sum())
p9_overall_accuracy = total_accuracy / (2 * len(decoding_results_df))



In [15]:
print(f"{p1_overall_accuracy}")
print(f"{p2_overall_accuracy}")
print(f"{p3_overall_accuracy}")
print(f"{p4_overall_accuracy}")
print(f"{p5_overall_accuracy}")
print(f"{p6_overall_accuracy}")
print(f"{p7_overall_accuracy}")
print(f"{p8_overall_accuracy}")
print(f"{p9_overall_accuracy}")

0.430225988700565
0.4344632768361582
0.481638418079096
0.4124293785310734
0.4401129943502825
0.42909604519774014
0.4652542372881356
0.44124293785310736
0.38305084745762713


In [16]:
print('anderson_baroni...DONE!')

anderson_baroni...DONE!
